1-create container
2- from storage accounts ---> Access control
3-Add ----> Role assignments ----> Storage Blob Data Contributor
4- tik ---->Managed identity
5- select member ----> Managed identity -----> Access Connector for Azure Databricks -----> select unity-catalog-access-connector
6-Databricks ---> catalog ----> External Data -----> Credential ----> create Credential ---> set name "datalake-db"---->cope and paste "resorce id unity-catalog-access"
7-Databricks ---> catalog ----> External Data ----->External location name "bronze"---> url ----> set Storage credential ""datalake-db""



In [0]:
# Mount ADLS Gen2
# Required each time the cluster is restarted which should be only on the first notebook as they run in order

tiers = ["bronze", "silver", "gold"]
adls_paths = [f"abfss://{tier}@rgdatabricksaccount.dfs.core.windows.net/" for tier in tiers]

# Accessing paths
bronze_adls = adls_paths[0]
silver_adls = adls_paths[1]
gold_adls = adls_paths[2]

dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)

[]

In [0]:
import requests
import json
from datetime import date, timedelta

In [0]:
start_date = date.today() - timedelta(days=1)
end_date = date.today()

In [0]:

# API را با تاریخ شروع و پایان مشخص شده توسط Data Factory فراخوانی کنید
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"

try:
    # Make the GET request to fetch data
    response = requests.get(url)

    # Check if the request was successful
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    data = response.json().get('features', [])

    if not data:
        print("No data returned for the specified date range.")
    else:
        # Specify the ADLS path
        file_path = f"{bronze_adls}/{start_date}_earthquake_data.json"

        # Save the JSON data
        json_data = json.dumps(data, indent=4)
        dbutils.fs.put(file_path, json_data, overwrite=True)
        print(f"Data successfully saved to file: {file_path}")

except requests.exceptions.RequestException as e:
    print(f"Error fetching data from API: {e}")



Wrote 403607 bytes.
Data successfully saved to file: abfss://bronze@rgdatabricksaccount.dfs.core.windows.net//2025-08-14_earthquake_data.json


In [0]:
data[0]

{'type': 'Feature',
 'properties': {'mag': 1.4,
  'place': '14 km WNW of Two Rivers, Alaska',
  'time': 1755215902817,
  'updated': 1755216036375,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ak025ae5hvt5',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ak025ae5hvt5&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 30,
  'net': 'ak',
  'code': '025ae5hvt5',
  'ids': ',ak025ae5hvt5,',
  'sources': ',ak,',
  'types': ',origin,phase-data,',
  'nst': None,
  'dmin': None,
  'rms': 0.7,
  'gap': None,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.4 - 14 km WNW of Two Rivers, Alaska'},
 'geometry': {'type': 'Point', 'coordinates': [-147.2951, 64.9394, 0]},
 'id': 'ak025ae5hvt5'}

In [0]:
# Define your variables
output_data = {
    "start_date": start_date.isoformat(),
    "end_date": end_date.isoformat(),
    "bronze_adls": bronze_adls,
    "silver_adls": silver_adls,
    "gold_adls": gold_adls,
}

# Return the dictionary directly
dbutils.jobs.taskValues.set(key="bronze_output", value=output_data)

In [0]:
output_data

{'start_date': '2025-08-14',
 'end_date': '2025-08-15',
 'bronze_adls': 'abfss://bronze@rgdatabricksaccount.dfs.core.windows.net/',
 'silver_adls': 'abfss://silver@rgdatabricksaccount.dfs.core.windows.net/',
 'gold_adls': 'abfss://gold@rgdatabricksaccount.dfs.core.windows.net/'}